# Heroes Of Plymoli : Purchasing Data Analysis

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

# lets see if any null values
purchase_data.isnull().values.any()

False

In [2]:
#lets review head of our datadrame
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


## Purchasing Analysis (Total)

* Obtaining number of unique items, average price and total revenue
* Creating a summary data frame to hold the results
* Giving the displayed data cleaner formatting as per requirement
* Display the summary data frame


In [3]:
# lets see unique values count in each column
purchase_data.nunique()

Purchase ID    780
SN             576
Age             39
Gender           3
Item ID        179
Item Name      179
Price          145
dtype: int64

In [4]:
# Converting unique values count array to dataframe as per specified format
summary_df=purchase_data.nunique().to_frame().swapaxes("index","columns").drop(["Age","Gender","Item ID"], axis=1)
summary_df=summary_df.rename(columns={"SN":"Total Players","Item Name":"Number of Unique Items","Purchase ID":"Number of Purchases","Price":"Average Price"})
#calculating & formating mean and total for purchase price
summary_df["Average Price"]=purchase_data["Price"].mean()
summary_df["Total Revenue"]=purchase_data["Price"].sum()
# format
f_summary_df=summary_df
f_summary_df["Average Price"]=f_summary_df["Average Price"].apply('${:,.2f}'.format)
f_summary_df["Total Revenue"]=f_summary_df["Total Revenue"].apply('${:,.2f}'.format)
f_summary_df 


,Number of Purchases,Total Players,Number of Unique Items,Average Price,Total Revenue
0,780,576,179,$3.05,"$2,379.77"


In [5]:
#re-arrange coulmns of summary as per requested order
f_summary_df=f_summary_df[["Number of Unique Items","Average Price","Number of Purchases", "Total Revenue"]]
f_summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Player Count

* Display the total number of players


In [6]:
# Total player count data frame  
#tp_df=summary_df.drop(["Number of Unique Items","Average Price","Number of Purchases", "Total Revenue"],axis=1)
tp_df=pd.DataFrame(summary_df["Total Players"])
tp_df

,Total Players
0,576


## Gender Demographics

* Percentage and Count of Male Players
* Percentage and Count of Female Players
* Percentage and Count of Other / Non-Disclosed

In [7]:
# Grouping original data for Gender column  , aggregating to get nice number of unique users count dataframe , sorting descending
gender_summary_df=purchase_data.groupby(["Gender"]).agg({"SN":'nunique'}).sort_values("SN",ascending=False)
gender_summary_df

,SN
Gender,
Male,484
Female,81
Other / Non-Disclosed,11


In [8]:
# renaming column 
gender_summary_df=gender_summary_df.rename(columns={"SN":"Total Count"})
gender_summary_df

,Total Count
Gender,
Male,484
Female,81
Other / Non-Disclosed,11


In [9]:
# adding column with % of gender
#gender_summary_df["Percentage of Players"]=[(row/tp_df["Total Players"][0]) for row in gender_summary_df["Total Count"]]
# i am a sucker for string comprehensions and LAMBDA ... but there is a simpler way to divide :) ... 
gender_summary_df["Percentage of Players"]=gender_summary_df["Total Count"]/tp_df["Total Players"][0]
#format
f_gender_summary_df=gender_summary_df
f_gender_summary_df["Percentage of Players"]=f_gender_summary_df["Percentage of Players"].apply('{:.2%}'.format)
f_gender_summary_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%



## Purchasing Analysis (Gender)

* Obtaining purchase count, avg. purchase price, avg. purchase total per person etc. by gender
* Creating a summary data frame to hold the results
* Giving the displayed data cleaner formatting
* Displaying the summary data frame

In [10]:
#lets Aggregate
avg_price_df=purchase_data.groupby(["Gender"]).agg({"Purchase ID":"count", "Price":['mean','sum']})
avg_price_df

Purchase ID     Price         
                            count      mean      sum
Gender                                              
Female                        113  3.203009   361.94
Male                          652  3.017853  1967.64
Other / Non-Disclosed          15  3.346000    50.19

In [11]:
# get rid of of multilevel columnn names 
avg_price_df.columns=["Purchase Count","Average Purchase Price","Total Purchase Value"]
avg_price_df

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,3.203009,361.94
Male,652,3.017853,1967.64
Other / Non-Disclosed,15,3.346000,50.19


In [12]:
# Adding Average spend by gender column
avg_price_df["Avg Total Purchase per Person"]=avg_price_df["Total Purchase Value"]/gender_summary_df["Total Count"]
avg_price_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [13]:
# format
f_avg_price_df=avg_price_df
f_avg_price_df[["Average Purchase Price","Avg Total Purchase per Person"]]=f_avg_price_df[["Average Purchase Price","Avg Total Purchase per Person"]].applymap(lambda x:f'${x:,.2f}')
f_avg_price_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,361.94,$4.47
Male,652,$3.02,1967.64,$4.07
Other / Non-Disclosed,15,$3.35,50.19,$4.56


## Age Demographics

* Establish bins for ages
* Categorize the existing players using the age bins. Hint: use pd.cut()
* Calculate the numbers and percentages by age group
* Create a summary data frame to hold the results
* Rounding the percentage column to two decimal points
* Display Age Demographics Table

In [14]:
# making bins and bins names
# age_bins=[i for i in range(9,41,5)]
age_bins=[0,9, 14, 19, 24, 29, 34, 39,200]
age_bins
bins_names=["<10",'10-14','15-19','20-24','25-29',"30-34",'35-39','40+'] 
bins_names

['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

In [15]:
#binning  age and adding column to original date frame
purchase_data["Age Ranges"]=pd.cut(purchase_data["Age"], age_bins,labels=bins_names,include_lowest=True)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price,Age Ranges
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40+
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [16]:
#grouping by AGES players count and getting %
age_summary_df=purchase_data.groupby(["Age Ranges"]).agg({"SN":'nunique'})
age_summary_df=age_summary_df.rename(columns={"SN":"Total Count"})
age_summary_df["Percentage of Players"]=age_summary_df["Total Count"]/tp_df["Total Players"][0]
#format
f_age_summary_df=age_summary_df
f_age_summary_df["Percentage of Players"]=f_age_summary_df["Percentage of Players"].apply('{:.2%}'.format)
f_age_summary_df

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age
* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below
* Create a summary data frame to hold the results
* Optional: give the displayed data cleaner formatting
* Display the summary data frame

In [17]:
# Grouping by age bins 
avg_age_df=purchase_data.groupby(["Age Ranges"]).agg({"Purchase ID":"count", "Price":['mean','sum']})
avg_age_df.columns=["Purchase Count","Average Purchase Price","Total Purchase Value"]
avg_price_df
avg_age_df["Avg Total Purchase per Person"]=avg_age_df["Total Purchase Value"]/age_summary_df["Total Count"]
avg_age_df


,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [18]:
# format 
f_avg_age_df=avg_age_df
f_avg_age_df[["Average Purchase Price","Avg Total Purchase per Person"]]=f_avg_age_df[["Average Purchase Price","Avg Total Purchase per Person"]].applymap(lambda x:f'${x:,.2f}')
f_avg_age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,77.13,$4.54
10-14,28,$2.96,82.78,$3.76
15-19,136,$3.04,412.89,$3.86
20-24,365,$3.05,1114.06,$4.32
25-29,101,$2.90,293.00,$3.81
30-34,73,$2.93,214.00,$4.12
35-39,41,$3.60,147.67,$4.76
40+,13,$2.94,38.24,$3.19


## Top Spenders

* Obtaining results for top spenders , count, average spend, total spend
* Create a summary data frame to hold the results
* Sort the total purchase value column in descending order
* Clean formatting as per request
* Display a preview of the summary data frame

In [19]:
# Grouping by player for count , average spend& total spend 
spenders_df=purchase_data.groupby(["SN"]).agg({"Purchase ID":"count", "Price":['mean','sum']})
spenders_df.columns=["Purchase Count","Average Purchase Price","Total Purchase Value"]
spenders_df=spenders_df.sort_values("Total Purchase Value",ascending=False)
spenders_df.head()
#formating
f_spenders_df=spenders_df
f_spenders_df[["Average Purchase Price","Total Purchase Value"]]=f_spenders_df[["Average Purchase Price","Total Purchase Value"]].applymap(lambda x:f'${x:,.2f}')
f_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Displaying most populat Item ID / Item Name vs purchase count, average item price, and total purchase value
* Sorting the purchase count column in descending order
* Clean formatting as per request
* Display a preview of the summary data frame

In [23]:
# Grouping by "Item ID"&"Item Name"for count , average spend& total spend , sort by count
item_df=purchase_data.groupby(["Item ID","Item Name"]).agg({"Purchase ID":"count", "Price":['mean','sum']})
item_df.columns=["Purchase Count","Item Price","Total Purchase Value"]
itempop_df=item_df.sort_values("Purchase Count",ascending=False)
#formating
f_itempop_df=itempop_df
f_itempop_df[["Item Price","Total Purchase Value"]]=f_itempop_df[["Item Price","Total Purchase Value"]].applymap(lambda x:f'${x:,.2f}')
f_itempop_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


# Most Profitable Items

* Sort the above table by total purchase value in descending order
* Clean formatting as per request
* Display a preview of the data frame

In [24]:
# resort by total value
itemprofit_df=item_df.sort_values("Total Purchase Value",ascending=False)
#formating
f_itemprofit_df=itemprofit_df
f_itemprofit_df[["Item Price","Total Purchase Value"]]=f_itemprofit_df[["Item Price","Total Purchase Value"]].applymap(lambda x:f'${x:,.2f}')
f_itemprofit_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


\pagebreak

# Heroes Of Plymoli : Purchasing Data Report

## Purchasing Analysis (Total)

##### "Heroes Of Plymoly" (HoG) game delivers total revenue of USD 2,379.77 from 576 gamers. Average spend per gamer is USD 3.05. 

In [25]:
f_summary_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,$3.05,780,"$2,379.77"


## Player Count

In [26]:
tp_df

,Total Players
0,576


## Gender Demographics

##### Gender demographics in HoG is heavily skewed to male players. Only 14 % of gamers are female .* Note to marketing department to review strategies to improve game's "gender diversity".

In [27]:
f_gender_summary_df

,Total Count,Percentage of Players
Gender,,
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


\pagebreak

## Purchasing Analysis (Gender)

##### In the table blow you can see in-game spend by gender. On average female players are spending 10% more than male.

In [28]:
avg_price_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,361.94,$4.47
Male,652,$3.02,1967.64,$4.07
Other / Non-Disclosed,15,$3.35,50.19,$4.56


## Age Demographics

##### Age Demographics is normally distributed around 20-24 age group with almost half of the players belonging to this group. 90% of players are from 15 to 40 years old.

In [29]:
f_age_summary_df

,Total Count,Percentage of Players
Age Ranges,,
<10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-39,31,5.38%
40+,12,2.08%


\pagebreak

## Purchasing Analysis (Age)

##### 35-40 age group is a biggest spender on average. Suprising runner up is less than 10 age group. 

In [30]:
f_avg_age_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Ranges,,,,
<10,23,$3.35,77.13,$4.54
10-14,28,$2.96,82.78,$3.76
15-19,136,$3.04,412.89,$3.86
20-24,365,$3.05,1114.06,$4.32
25-29,101,$2.90,293.00,$3.81
30-34,73,$2.93,214.00,$4.12
35-39,41,$3.60,147.67,$4.76
40+,13,$2.94,38.24,$3.19


## Top Spenders

##### List of top spenders in HoG.

In [31]:
f_spenders_df.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


\pagebreak

## Most Popular Items

##### List of most popular items in HoG

In [32]:
f_itempop_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


# Most Profitable Items

##### List of most profitable items in HoG

In [33]:
f_itemprofit_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
103,Singed Scalpel,8,$4.35,$34.80


##### The End